In [2]:
### PREREQUISTIES
### (many used only in one notebook...)

import os
import pandas as pd
import numpy as np
import logging ### to monitor the code
from bs4 import BeautifulSoup
import pickle
import math
import random
import sys
import csv
import unicodedata
import requests
import re

import nltk
from nltk.collocations import *

import networkx as nx

import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats

import xml.etree.cElementTree as ET
from urllib.request import urlopen

import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
#from google.colab import auth
#from oauth2client.client import GoogleCredentials
from google.oauth2 import service_account # based on google-auth library
import sddk

In [3]:
#from anda import gr
# OR uncomment the following:
script_url = "https://raw.githubusercontent.com/sdam-au/anda_py/master/anda/gr.py"
exec(requests.get(script_url).content)

In [4]:
### not neccessary for reading the data, just for exporting them to sciencedata.dk
conf = sddk.configure()

sciencedata.dk username (format '123456@au.dk'): 648597@au.dk
sciencedata.dk password: ········
endpoint variable has been configured to: https://sciencedata.dk/files/


In [5]:
# to access gsheet, you need Google Service Account key json file
# I have mine located in my personal space on sciencedata.dk, so I read it from there:

# (1) read the file and parse its content
file_data = conf[0].get(conf[1] + "ServiceAccountsKey.json").json()
# (2) transform the content into crendentials object
credentials = service_account.Credentials.from_service_account_info(file_data)
# (3) specify your usage of the credentials
scoped_credentials = credentials.with_scopes(['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive'])
# (4) use the constrained credentials for authentication of gspread package
gc = gspread.Client(auth=scoped_credentials)
# (5) establish connection with spreadsheets specified by their url
PIA_data = gc.open_by_url("https://docs.google.com/spreadsheets/d/1KxOx7Be9fj3lDcEPgQhQ-Iqcn9p367-MMD6RMXe8rks/edit?usp=sharing")
PIA_overview = gc.open_by_url("https://docs.google.com/spreadsheets/d/1e94wyelg6dftQ4zxbq1xvwxWAI-BhcYXtclDW-YTnrw/edit?usp=sharing")

In [6]:
### our data are located within a publicly available folder on sciencedata.dk
publicfolder = "31b393e2afe1ee96ce81869c7efe18cb"
c_aristotelicum = sddk.read_file("c_aristotelicum.json", "df", publicfolder)
c_hippocraticum = sddk.read_file("c_hippocraticum.json", "df", publicfolder)

reading file located in a public folder
reading file located in a public folder


In [7]:
# look at first 5 rows
c_aristotelicum.head(5)

,filename,author,title,string,wordcount,author_id,doc_id,raw_date,date_avr,date_probs,date_manual,provenience,lemmata,lemmata_wordcount,lemmatized_sentences,n_sentences,tlg_epithet
685,tlg0086.tlg001.1st1K-grc2.xml,Aristotle,Aristotelis Analytica Priora et Posteriora,Πρῶτον εἰπεῖν περὶ τί καὶ τίνος ἐστὶν ἡ σκέψις...,59754,tlg0086,tlg0086.tlg001,4 B.C.,-3.5,{'-3.5': 1},-3.5,pagan,"[πρότερος, εἶπον, σκέψις, ἀπόδειξις, ἐπιστήμη,...",21172,"[[πρότερος, εἶπον, σκέψις, ἀπόδειξις, ἐπιστήμη...",3643,Philosophici/-ae
686,tlg0086.tlg002.1st1K-grc2.xml,Aristotle,De anima,Τῶν καλῶν καὶ τιμίων τὴν εἴδησιν ὑπολαμβάνοντε...,20981,tlg0086,tlg0086.tlg002,4 B.C.,-3.5,{'-3.5': 1},-3.5,pagan,"[κᾶλον, τίμιος, εἶδος, μαλλός, ἕτερος, ἕτερος,...",8899,"[[κᾶλον, τίμιος, εἶδος, μαλλός, ἕτερος, ἕτερος...",1306,Philosophici/-ae
688,tlg0086.tlg005.1st1K-grc1.xml,Aristotle,De caelo,Ἡ περὶ φύσεως ἐπιστήμη σχεδὸν ἡ πλείστη φαίνετ...,31292,tlg0086,tlg0086.tlg005,4 B.C.,-3.5,{'-3.5': 1},-3.5,pagan,"[φύσις, ἐπιστήμη, πλεῖστος, φαίνω, σῶμα, μέγεθ...",12891,"[[φύσις, ἐπιστήμη, πλεῖστος, φαίνω, σῶμα, μέγε...",1973,Philosophici/-ae
689,tlg0086.tlg006.1st1K-grc1.xml,Aristotle,Categoriae,"ὉΜΩΝΥΜΑ λέγεται ὧν ὄνομα μόνον κοινόν, ὁ δὲ κα...",10316,tlg0086,tlg0086.tlg006,4 B.C.,-3.5,{'-3.5': 1},-3.5,pagan,"[ὁμωνυμα, λέγω, ὄνομα, μόνος, κοινός, ὄνομα, λ...",4330,"[[ὁμωνυμα, λέγω, ὄνομα, μόνος, κοινός, ὄνομα, ...",657,Philosophici/-ae
690,tlg0086.tlg008.1st1K-grc1.xml,Aristotle,De divinatione per somnum,Περὶ δὲ τῆς μαντικῆς τῆς ἐν τοῖς ὕπνοις γινομέ...,1194,tlg0086,tlg0086.tlg008,4 B.C.,-3.5,{'-3.5': 1},-3.5,pagan,"[μαντικός, ὕπνος, συμβαίνω, ἐνύπνιον, καταφρον...",515,"[[μαντικός, ὕπνος, συμβαίνω, ἐνύπνιον, καταφρο...",68,Philosophici/-ae


# Exploring Nicomachean Ethics
feel free to skip this section

In [8]:
EN_df = c_aristotelicum[c_aristotelicum["doc_id"]=="tlg0086.tlg010"]
EN_df

,filename,author,title,string,wordcount,author_id,doc_id,raw_date,date_avr,date_probs,date_manual,provenience,lemmata,lemmata_wordcount,lemmatized_sentences,n_sentences,tlg_epithet
692,tlg0086.tlg010.perseus-grc1.xml,Aristotle,Nicomachean Ethics,"πᾶσα τέχνη καὶ πᾶσα μέθοδος, ὁμοίως δὲ πρᾶξίς ...",56630,tlg0086,tlg0086.tlg010,4 B.C.,-3.5,{'-3.5': 1},-3.5,pagan,"[πᾶς, τέχνη, πᾶς, μέθοδος, ὅμοιος, πρᾶξίς, προ...",23811,"[[πᾶς, τέχνη, πᾶς, μέθοδος, ὅμοιος, πρᾶξίς, πρ...",3897,Philosophici/-ae


In [9]:
EN_df["string"].tolist()[0][:100]

'πᾶσα τέχνη καὶ πᾶσα μέθοδος, ὁμοίως δὲ πρᾶξίς τε καὶ προαίρεσις, ἀγαθοῦ τινὸς ἐφίεσθαι δοκεῖ· διὸ κα'

In [10]:
print(EN_df["lemmatized_sentences"].tolist()[0][:10])

[['πᾶς', 'τέχνη', 'πᾶς', 'μέθοδος', 'ὅμοιος', 'πρᾶξίς', 'προαίρεσις', 'ἀγαθός', 'ἐφίημι', 'δοκέω'], ['καλός', 'ἀποφαίνω', 'ἀγαθός', 'ἵημι', 'πάντʼ', 'ἐφίημι'], ['διαφορά', 'φαίνω', 'τέλος'], ['εἶμι', 'ἐνέργεια', 'παρʼ', 'ἔργον'], ['εἶμι', 'τέλος', 'πρᾶξις', 'βελτίων', 'φύω', 'ἐνέργεια', 'ἔργον'], ['πολύς', 'πρᾶξις', 'τέχνη', 'ἐπιστήμη', 'πολύς', 'γίγνομαι', 'τέλος'], ['ἰατρικός', 'ὑγίεια', 'ναυπηγικός', 'πλοῖον', 'στρατηγικός', 'νίκη', 'οἰκονομικός', 'πλοῦτος'], ['ὅσος', 'εἶμι', 'μίαν', 'δύναμις', 'ἱππικην', 'χαλινοποιική', 'ὅσος', 'ἱππικός', 'ὄργανον', 'εἶμι', 'πᾶς', 'πολεμικός', 'πρᾶξις', 'στρατηγικός', 'τρόπος', 'ὑφʼ', 'ἕτερος'], ['ἅπας', 'ἀρχιτεκτονικῶν', 'τέλος', 'πᾶς', 'αἱρετός', 'ὑπʼ'], ['χάρις', 'ἐκεῖνος', 'διώκω']]


In [11]:
EN_df["lemmatized_sentences"].tolist()[0][:5]

[['πᾶς',
  'τέχνη',
  'πᾶς',
  'μέθοδος',
  'ὅμοιος',
  'πρᾶξίς',
  'προαίρεσις',
  'ἀγαθός',
  'ἐφίημι',
  'δοκέω'],
 ['καλός', 'ἀποφαίνω', 'ἀγαθός', 'ἵημι', 'πάντʼ', 'ἐφίημι'],
 ['διαφορά', 'φαίνω', 'τέλος'],
 ['εἶμι', 'ἐνέργεια', 'παρʼ', 'ἔργον'],
 ['εἶμι', 'τέλος', 'πρᾶξις', 'βελτίων', 'φύω', 'ἐνέργεια', 'ἔργον']]

In [12]:
" ".join(EN_df["lemmatized_sentences"].tolist()[0][0])

'πᾶς τέχνη πᾶς μέθοδος ὅμοιος πρᾶξίς προαίρεσις ἀγαθός ἐφίημι δοκέω'

In [13]:
EN_sentences = ""
for sentence in EN_df["lemmatized_sentences"].tolist()[0]:
  EN_sentences = EN_sentences + " ".join(sentence) + ". "

In [14]:
EN_sentences[:1000]

'πᾶς τέχνη πᾶς μέθοδος ὅμοιος πρᾶξίς προαίρεσις ἀγαθός ἐφίημι δοκέω. καλός ἀποφαίνω ἀγαθός ἵημι πάντʼ ἐφίημι. διαφορά φαίνω τέλος. εἶμι ἐνέργεια παρʼ ἔργον. εἶμι τέλος πρᾶξις βελτίων φύω ἐνέργεια ἔργον. πολύς πρᾶξις τέχνη ἐπιστήμη πολύς γίγνομαι τέλος. ἰατρικός ὑγίεια ναυπηγικός πλοῖον στρατηγικός νίκη οἰκονομικός πλοῦτος. ὅσος εἶμι μίαν δύναμις ἱππικην χαλινοποιική ὅσος ἱππικός ὄργανον εἶμι πᾶς πολεμικός πρᾶξις στρατηγικός τρόπος ὑφʼ ἕτερος. ἅπας ἀρχιτεκτονικῶν τέλος πᾶς αἱρετός ὑπʼ. χάρις ἐκεῖνος διώκω. διαφέρω ἐνέργεια τέλος πρᾶξις ἐπιστήμη. τέλος πρακτός διʼ βούλομαι πᾶς διʼ ἕτερος αἱρέω ʽπρόεισι γʼ ἄπειρος ὥστʼ κενός μάταιος ὄρεξις δῆλος τοῦτʼ ἐάω ἀγαθός ἄριστον. ἆρʼ βιός γνῶσις μέγας ἔχις ῥοπή τοξότης σκοπός μαλλός τυγχάνω δέον. πειρατέος τύπος περιλαμβάνω ποτʼ ἐπιστήμη δύναμις. δοκέω κύριος ἀρχιτεκτονικῆς. πολιτικός φαίνω. χρεὼν ἐπιστήμη πόλις πόα ἕκαστος μανθάνω διατάσσω. ὁράω ἔντιμος δύναμις οἷος στρατηγικός οἰκονομικην ῥητορικός. λοιπός πρακτικός ἐπιστήμη δεῖ πράσσω ἀπέχω τέλ

In [15]:
file_object = open("EN_sentences.txt","w") 
file_object.write(EN_sentences)

172282

In [16]:
print(EN_df["lemmata"].tolist()[0][:100])

['πᾶς', 'τέχνη', 'πᾶς', 'μέθοδος', 'ὅμοιος', 'πρᾶξίς', 'προαίρεσις', 'ἀγαθός', 'ἐφίημι', 'δοκέω', 'καλός', 'ἀποφαίνω', 'ἀγαθός', 'ἵημι', 'πάντʼ', 'ἐφίημι', 'διαφορά', 'φαίνω', 'τέλος', 'εἶμι', 'ἐνέργεια', 'παρʼ', 'ἔργον', 'εἶμι', 'τέλος', 'πρᾶξις', 'βελτίων', 'φύω', 'ἐνέργεια', 'ἔργον', 'πολύς', 'πρᾶξις', 'τέχνη', 'ἐπιστήμη', 'πολύς', 'γίγνομαι', 'τέλος', 'ἰατρικός', 'ὑγίεια', 'ναυπηγικός', 'πλοῖον', 'στρατηγικός', 'νίκη', 'οἰκονομικός', 'πλοῦτος', 'ὅσος', 'εἶμι', 'μίαν', 'δύναμις', 'ἱππικην', 'χαλινοποιική', 'ὅσος', 'ἱππικός', 'ὄργανον', 'εἶμι', 'πᾶς', 'πολεμικός', 'πρᾶξις', 'στρατηγικός', 'τρόπος', 'ὑφʼ', 'ἕτερος', 'ἅπας', 'ἀρχιτεκτονικῶν', 'τέλος', 'πᾶς', 'αἱρετός', 'ὑπʼ', 'χάρις', 'ἐκεῖνος', 'διώκω', 'διαφέρω', 'ἐνέργεια', 'τέλος', 'πρᾶξις', 'ἐπιστήμη', 'τέλος', 'πρακτός', 'διʼ', 'βούλομαι', 'πᾶς', 'διʼ', 'ἕτερος', 'αἱρέω', 'ʽπρόεισι', 'γʼ', 'ἄπειρος', 'ὥστʼ', 'κενός', 'μάταιος', 'ὄρεξις', 'δῆλος', 'τοῦτʼ', 'ἐάω', 'ἀγαθός', 'ἄριστον', 'ἆρʼ', 'βιός', 'γνῶσις', 'μέγας']


# Key term replacement

In [17]:
# manually define key terms we are interested in:
keyterm_patterns = [("^λ[υ|ύ]π.+", "λύπ*"), ("^[ἀ|ἄ]λγ.+", "ἄλγ*"), ("^[ὀ|ὠ]δ[ύ|υ]ν.", "ὀδύν*"), ("^π[ό|ο]ν[ο|ό|έ|ε|η|ή|ῆ](?!ρ).+", "πόνο*")]

In [18]:
### produce a list of all words from the authors
### (useful for a preliminary exploration)
aristotle_list = []
for list_element in c_aristotelicum["lemmata"].tolist():
  aristotle_list.extend(list_element)
hippocrates_list = []
for list_element in c_hippocraticum["lemmata"].tolist():
  hippocrates_list.extend(list_element)
one_merged_list = aristotle_list + hippocrates_list

In [20]:
# unique word forms
matches = []
for pattern_tuple in keyterm_patterns:
  r = re.compile(pattern_tuple[0])
  matches.extend([(pattern_tuple[0], pattern_tuple[1], match, aristotle_list.count(match), hippocrates_list.count(match), lemma_translator(match)) for match in list(filter(r.match, list(set(aristotle_list + hippocrates_list))))])
matches_df = pd.DataFrame(matches, columns=["pattern", "replacement", "match", "c_aristotelicum", "c_hippocraticum", "translation"])
matches_df

,pattern,replacement,match,c_aristotelicum,c_hippocraticum,translation
0,^λ[υ|ύ]π.+,λύπ*,λυπέον,0,8,
1,^λ[υ|ύ]π.+,λύπ*,λυπεῖταἰ,1,0,
2,^λ[υ|ύ]π.+,λύπ*,λυπέουσα,0,1,
3,^λ[υ|ύ]π.+,λύπ*,λυπηρότερά,0,1,
4,^λ[υ|ύ]π.+,λύπ*,λυπέεται,0,3,
...,...,...,...,...,...,...
119,^π[ό|ο]ν[ο|ό|έ|ε|η|ή|ῆ](?!ρ).+,πόνο*,πονεούσης,0,1,
120,^π[ό|ο]ν[ο|ό|έ|ε|η|ή|ῆ](?!ρ).+,πόνο*,πονῆται,0,1,
121,^π[ό|ο]ν[ο|ό|έ|ε|η|ή|ῆ](?!ρ).+,πόνο*,πονοῦσαι,1,0,
122,^π[ό|ο]ν[ο|ό|έ|ε|η|ή|ῆ](?!ρ).+,πόνο*,πονεομένου,0,1,


In [21]:
### look at all unique matched words
print(matches_df["match"].tolist())

['λυπέον', 'λυπεῖταἰ', 'λυπέουσα', 'λυπηρότερά', 'λυπέεται', 'λυπή', 'λύπαςοὐ', 'λυπέουσιν', 'λυπέωσι', 'λυπηρός', 'λύπημα', 'λυπητικός', 'λυπέοντος', 'λυπέουσαν', 'λυποδίαν', 'λυπρᾷ', 'λυπεόντων', 'λυπέοιτο', 'λυπηθῆ', 'λύπη', 'λυπούμενός', 'λυπέονταί', 'λυπέῃ', 'λυπέοντα', 'λύπαςμεσότης', 'λυπηθήσοιτʼ', 'λυπεῖταί', 'λυπέω', 'ἀλγέοιεν', 'ἄλγημά', 'ἄλγλος', 'ἀλγέοντες', 'ἀλγεούσης', 'ἀλγηδών', 'ἀλγέουσι', 'ἀλγεινην', 'ἀλγέουσαν', 'ἀλγηδὼν', 'ἀλγέοντάς', 'ἀλγέοντι', 'ἀλγεόντων', 'ἄλγησις', 'ἀλγήμασι', 'ἀλγέον', 'ἀλγήσῃ', 'ἄλγημα', 'ἀλγέειν', 'ἀλγέοντας', 'ἀλγέοντα', 'ἀλγέουσιν', 'ἀλγήσωσιν', 'ἀλγέω', 'ἀλγέωσι', 'ἀλγέοντος', 'ἄλγεσκ', 'ἄλγος', 'ἀλγέῃ', 'ἀλγεῦντα', 'ἀλγήσαν', 'ἀλγεινός', 'ἀλγεούσῃσι', 'ὀδυνῶδες', 'ὠδυναμίη', 'ὀδυνῇ', 'ὀδυνήματα', 'ὀδυνωμένους', 'ὀδυνῶνται', 'ὀδυναίτερον', 'ὀδυνήμασι', 'ὀδυνωμένῳ', 'ὀδυνέων', 'ὀδυνώμενα', 'ὀδυνης', 'ὀδυνηθῇ', 'ὀδυνηρός', 'ὀδυνώδης', 'ὀδύνη', 'ὀδυνημάτων', 'ὀδύνῃσί', 'ὀδύνατον', 'ὀδυνέωνται', 'ὀδυνώδεας', 'ὀδυνώδεες', 'ὀδυνάω', 'ὀδυνώδεα', 

In [22]:
# uncomment the following to export to gsheet: 
set_with_dataframe(PIA_overview.add_worksheet("matches_20201027", 1, 1), matches_df)

In [23]:
# make 
def replacer_word_list(pattern, product, word_list):
  return [re.sub(pattern, product, word) for word in word_list]

def replace_keywords(list_of_words, list_of_tuples):
  for pattern in list_of_tuples:
    list_of_words = replacer_word_list(pattern[0], pattern[1], list_of_words)
  return list_of_words

In [24]:
# test 1:
testlist = ["μέλυπρᾷ", "λυπρᾷ"] 
for word in testlist:
  print(re.sub(keyterm_patterns[0][0], keyterm_patterns[0][1], word))

μέλυπρᾷ
λύπ*


In [25]:
### test 2:
word_list_test = ['βοοκ', 'πᾶς', 'μέλυπρᾷ', "ἄλγτέχνη",'τέχνη' ,'πᾶς', 'μέθοδος', 'ὅμοιος', "λύπη",'πρᾶξίς', 'προαίρεσις', 'ἀγαθός', 'ἐφίημι']
replace_keywords(word_list_test, keyterm_patterns)

['βοοκ',
 'πᾶς',
 'μέλυπρᾷ',
 'ἄλγ*',
 'τέχνη',
 'πᾶς',
 'μέθοδος',
 'ὅμοιος',
 'λύπ*',
 'πρᾶξίς',
 'προαίρεσις',
 'ἀγαθός',
 'ἐφίημι']

In [26]:
# apply the replacement on the level of individual words
c_aristotelicum["lemmata_repl"] = c_aristotelicum["lemmata"].apply(lambda x: replace_keywords(x, keyterm_patterns))
c_hippocraticum["lemmata_repl"] = c_hippocraticum["lemmata"].apply(lambda x: replace_keywords(x, keyterm_patterns))

In [27]:
# apply the replacement on the sentences
def replace_in_sentences(list_of_sentences):
    return [replace_keywords(sentence, keyterm_patterns) for sentence in list_of_sentences]

c_aristotelicum["lemmatized_sentences_repl"] = c_aristotelicum["lemmatized_sentences"].apply(replace_in_sentences)
c_hippocraticum["lemmatized_sentences_repl"] = c_hippocraticum["lemmatized_sentences"].apply(replace_in_sentences)

In [28]:
c_aristotelicum.head(5)

,filename,author,title,string,wordcount,author_id,doc_id,raw_date,date_avr,date_probs,date_manual,provenience,lemmata,lemmata_wordcount,lemmatized_sentences,n_sentences,tlg_epithet,lemmata_repl,lemmatized_sentences_repl
685,tlg0086.tlg001.1st1K-grc2.xml,Aristotle,Aristotelis Analytica Priora et Posteriora,Πρῶτον εἰπεῖν περὶ τί καὶ τίνος ἐστὶν ἡ σκέψις...,59754,tlg0086,tlg0086.tlg001,4 B.C.,-3.5,{'-3.5': 1},-3.5,pagan,"[πρότερος, εἶπον, σκέψις, ἀπόδειξις, ἐπιστήμη,...",21172,"[[πρότερος, εἶπον, σκέψις, ἀπόδειξις, ἐπιστήμη...",3643,Philosophici/-ae,"[πρότερος, εἶπον, σκέψις, ἀπόδειξις, ἐπιστήμη,...","[[πρότερος, εἶπον, σκέψις, ἀπόδειξις, ἐπιστήμη..."
686,tlg0086.tlg002.1st1K-grc2.xml,Aristotle,De anima,Τῶν καλῶν καὶ τιμίων τὴν εἴδησιν ὑπολαμβάνοντε...,20981,tlg0086,tlg0086.tlg002,4 B.C.,-3.5,{'-3.5': 1},-3.5,pagan,"[κᾶλον, τίμιος, εἶδος, μαλλός, ἕτερος, ἕτερος,...",8899,"[[κᾶλον, τίμιος, εἶδος, μαλλός, ἕτερος, ἕτερος...",1306,Philosophici/-ae,"[κᾶλον, τίμιος, εἶδος, μαλλός, ἕτερος, ἕτερος,...","[[κᾶλον, τίμιος, εἶδος, μαλλός, ἕτερος, ἕτερος..."
688,tlg0086.tlg005.1st1K-grc1.xml,Aristotle,De caelo,Ἡ περὶ φύσεως ἐπιστήμη σχεδὸν ἡ πλείστη φαίνετ...,31292,tlg0086,tlg0086.tlg005,4 B.C.,-3.5,{'-3.5': 1},-3.5,pagan,"[φύσις, ἐπιστήμη, πλεῖστος, φαίνω, σῶμα, μέγεθ...",12891,"[[φύσις, ἐπιστήμη, πλεῖστος, φαίνω, σῶμα, μέγε...",1973,Philosophici/-ae,"[φύσις, ἐπιστήμη, πλεῖστος, φαίνω, σῶμα, μέγεθ...","[[φύσις, ἐπιστήμη, πλεῖστος, φαίνω, σῶμα, μέγε..."
689,tlg0086.tlg006.1st1K-grc1.xml,Aristotle,Categoriae,"ὉΜΩΝΥΜΑ λέγεται ὧν ὄνομα μόνον κοινόν, ὁ δὲ κα...",10316,tlg0086,tlg0086.tlg006,4 B.C.,-3.5,{'-3.5': 1},-3.5,pagan,"[ὁμωνυμα, λέγω, ὄνομα, μόνος, κοινός, ὄνομα, λ...",4330,"[[ὁμωνυμα, λέγω, ὄνομα, μόνος, κοινός, ὄνομα, ...",657,Philosophici/-ae,"[ὁμωνυμα, λέγω, ὄνομα, μόνος, κοινός, ὄνομα, λ...","[[ὁμωνυμα, λέγω, ὄνομα, μόνος, κοινός, ὄνομα, ..."
690,tlg0086.tlg008.1st1K-grc1.xml,Aristotle,De divinatione per somnum,Περὶ δὲ τῆς μαντικῆς τῆς ἐν τοῖς ὕπνοις γινομέ...,1194,tlg0086,tlg0086.tlg008,4 B.C.,-3.5,{'-3.5': 1},-3.5,pagan,"[μαντικός, ὕπνος, συμβαίνω, ἐνύπνιον, καταφρον...",515,"[[μαντικός, ὕπνος, συμβαίνω, ἐνύπνιον, καταφρο...",68,Philosophici/-ae,"[μαντικός, ὕπνος, συμβαίνω, ἐνύπνιον, καταφρον...","[[μαντικός, ὕπνος, συμβαίνω, ἐνύπνιον, καταφρο..."


In [29]:
sddk.write_file("public_data/PIA/c_aristotelicum_repl.json", c_aristotelicum, conf)
sddk.write_file("public_data/PIA/c_hippocraticum_repl.json", c_hippocraticum, conf)

A file with the same name ("c_aristotelicum_repl.json") already exists in this location.
Press Enter to overwrite it or choose different path and filename: 
Your <class 'pandas.core.frame.DataFrame'> object has been succefully written as "https://sciencedata.dk/files/public_data/PIA/c_aristotelicum_repl.json"
A file with the same name ("c_hippocraticum_repl.json") already exists in this location.
Press Enter to overwrite it or choose different path and filename: 
Your <class 'pandas.core.frame.DataFrame'> object has been succefully written as "https://sciencedata.dk/files/public_data/PIA/c_hippocraticum_repl.json"
